In [52]:
from os import name
import requests
import fake_useragent
from bs4 import BeautifulSoup
import time
import json

In [53]:
def get_links(text):
    ua =fake_useragent.UserAgent()
    data = requests.get(
        url=f"https://ekaterinburg.hh.ru/search/vacancy?text={text}&salary=&area=3&ored_clusters=true&enable_snippets=true&page=1",
        headers={"user-agent":ua.random}
    )
    if data.status_code != 200:
        return
    soup = BeautifulSoup(data.content, "lxml")
    try:
        page_count = int(soup.find("div",attrs={"class":"pager"}).find_all("span",recursive=False)[-1].find("a").find("span").text)
    except:
        return
    for page in range(page_count):
        try:
            data = requests.get(
            url=f"https://ekaterinburg.hh.ru/search/vacancy?text={text}&salary=&area=3&ored_clusters=true&enable_snippets=true&page={page}",
            headers={"user-agent":ua.random}
            )
            if data.status_code != 200:
                continue
            soup = BeautifulSoup(data.content, "lxml")
            for a in soup.find_all("a",attrs={"class":"serp-item__title"}):
                yield f"https://hh.ru{a.attrs['href'].split('?')[0]}"
        except Exception as e:
            print(f"{e}")
        time.sleep(1)
   

In [54]:
def get_workaday(link):
    ua =fake_useragent.UserAgent()
    data = requests.get(
        url=link,
        headers = {"user-agent":ua.random}
    )
    if data.status_code != 200:
        return
    soup = BeautifulSoup(data.content, "lxml")
    try:
        name = soup.find(attrs={"class":"bloko-header-section-1"}).text
    except:
        name = ""
    try:
        salary = soup.find(attrs={"class":"bloko-header-section-2_lite"}).text
    except:
        salary = ""    
    workaday = {
        "name":name,
        "salary":salary
    }
    return workaday
    

In [55]:
if __name__ == "__main__":
    data = []
    for a in get_links("python"):
        data.append(get_workaday(a))
        time.sleep(1)
        with open("data.json", "w", encoding="utf-8") as f:
            json.dump(data,f,indent=4,ensure_ascii=False)
    

ConnectionError: HTTPSConnectionPool(host='hh.ruhttps', port=443): Max retries exceeded with url: //ekaterinburg.hh.ru/vacancy/77146976 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021C31525690>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))